In [1]:
import os
import shutil
import subprocess
import zipfile
from PIL import Image, ImageDraw, ImageFont

from icecream import ic

from collage import make_diff_image

In [19]:
def extract(path_to_zip, target_dir):
    with zipfile.ZipFile(path_to_zip, 'r') as target_zip:
        target_zip.extractall(target_dir)
        
        
def find_main_tex(path):
    directory = os.path.join(os.getcwd(), path)
    tex_file = None
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tex'):
                tex_file = os.path.join(root, file)
                break
        if tex_file:
            break
    return tex_file


def get_image_paths(target_tex_file):
    image_paths = []
    with open(target_tex_file, 'r') as tex_file:
        for line in tex_file:
            if not line.strip().startswith('%') and 'includegraphics' in line:
                start_index = line.find('{') + 1
                end_index = line.find('}')
                image_path = line[start_index:end_index]
                image_paths.append(image_path)
    return image_paths


def find_changed_images(old_image_paths, new_image_paths):
    """
        for the input
            old_image_paths = ['fig1', 'fig2', 'fig3', 'fig4', 'fig5']
            new_image_paths = ['fig1', 'fig2_new', 'fig3', 'fig4', 'fig5', 'fig6']
        the return would be
            [False, True, False, False, False, True]
    """
    changed_images = [True] * len(new_image_paths)

    for index, new_imag in enumerate(new_image_paths):
        if index < len(old_image_paths):
            if old_image_paths[index] == new_imag:
                changed_images[index] = False
                
    return changed_images


def make_all_collages(old_image_paths_full, new_image_paths_full, changed_images):
    for i, new_image_path in enumerate(new_image_paths):
        if changed_images[i]:
            if i < len(old_image_paths_full):
                make_diff_image(
                    path_img_old=old_image_paths_full[i], 
                    path_img_new=new_image_paths_full[i], 
                    path_img_diff=new_image_paths_full[i]
                )
            else:
                make_diff_image(
                    path_img_old=blank_image_path, 
                    path_img_new=new_image_paths_full[i], 
                    path_img_diff=new_image_paths_full[i]
                )
                
def create_diffpdf(old_tex_file, new_tex_file, dir_new_full):
    diff_file = 'diff.tex'
    latexdiff_command = f'latexdiff {old_tex_file} {new_tex_file} > {dir_new_full}/{diff_file}'
    subprocess.run(
        latexdiff_command, 
        shell=True, 
        #check=True
    )
    
    os.chdir(dir_new_full)
    
    # Compile the diff.tex file
    latexmk_command = f'latexmk -f -pdf {diff_file}'
    subprocess.run(
        latexmk_command, 
        shell=True, 
        #check=True
    )

    os.chdir('..')
    
    shutil.move('new/diff.pdf', 'diff.pdf')

In [ ]:
def make_diffpdf(old_zip, new_zip):
    dir_new = 'new'
    dir_old = 'old'
    
    dir_new_full = os.path.join(os.getcwd(), dir_new)
    dir_old_full = os.path.join(os.getcwd(), dir_old)

    extract(new_zip, dir_new)
    extract(old_zip, dir_old)

    old_tex_file = find_main_tex(dir_old)
    new_tex_file = find_main_tex(dir_new)
    
    old_image_paths = get_image_paths(old_tex_file)
    new_image_paths = get_image_paths(new_tex_file)
    
    changed_images = find_changed_images(old_image_paths, new_image_paths)
    
    
    new_image_paths_full = []
    for i in range(len(new_image_paths)):
        new_image_paths_full.append(os.path.join(dir_new_full, new_image_paths[i]))

    old_image_paths_full = []
    for i in range(len(old_image_paths)):
        old_image_paths_full.append(os.path.join(dir_old_full, old_image_paths[i]))

    blank_image_path = os.path.join(os.getcwd(), 'blank.jpg')
    
    
    make_all_collages(old_image_paths_full, new_image_paths_full, changed_images)
    
    create_diffpdf(old_tex_file, new_tex_file, dir_new_full)
    
    # Clean up temporary directories and files
    shutil.rmtree(dir_new_full)
    shutil.rmtree(dir_old_full)

In [3]:
new_zip = 'new.zip'
old_zip = 'old.zip'

dir_new = 'new'
dir_old = 'old'

dir_new_full = os.path.join(os.getcwd(), dir_new)
dir_old_full = os.path.join(os.getcwd(), dir_old)

extract(new_zip, dir_new)
extract(old_zip, dir_old)

print(dir_new_full, '\n', dir_old_full)

/home/ivan/Documents/GitHub/latexdiff-bot/test/new 
 /home/ivan/Documents/GitHub/latexdiff-bot/test/old


In [4]:
old_tex_file = find_main_tex(dir_old)
new_tex_file = find_main_tex(dir_new)

print(old_tex_file, '\n', new_tex_file)

/home/ivan/Documents/GitHub/latexdiff-bot/test/old/main.tex 
 /home/ivan/Documents/GitHub/latexdiff-bot/test/new/main.tex


In [5]:
old_image_paths = get_image_paths(old_tex_file)
new_image_paths = get_image_paths(new_tex_file)

print(old_image_paths, '\n', new_image_paths)

['fig/figure1.png'] 
 ['fig/image.png', 'fig/E_2_2.png']


In [6]:
changed_images = find_changed_images(old_image_paths, new_image_paths)
changed_images

[True, True]

In [7]:
new_image_paths_full = []
for i in range(len(new_image_paths)):
    new_image_paths_full.append(os.path.join(dir_new_full, new_image_paths[i]))
    
old_image_paths_full = []
for i in range(len(old_image_paths)):
    old_image_paths_full.append(os.path.join(dir_old_full, old_image_paths[i]))
    
blank_image_path = os.path.join(os.getcwd(), 'blank.jpg')

In [8]:
ic(
    new_image_paths_full,
    old_image_paths_full,
    blank_image_path
);

ic| new_image_paths_full: ['/home/ivan/Documents/GitHub/latexdiff-bot/test/new/fig/image.png',
                           '/home/ivan/Documents/GitHub/latexdiff-bot/test/new/fig/E_2_2.png']
    old_image_paths_full: ['/home/ivan/Documents/GitHub/latexdiff-bot/test/old/fig/figure1.png']
    blank_image_path: '/home/ivan/Documents/GitHub/latexdiff-bot/test/blank.jpg'


In [10]:
make_all_collages(old_image_paths_full, new_image_paths_full, changed_images)

In [18]:
create_diffpdf(old_tex_file, new_tex_file, dir_new_full)

Rc files read:
  /etc/latexmkrc
Latexmk: This is Latexmk, John Collins, 4 Apr. 2023. Version 4.80.
Latexmk: applying rule 'pdflatex'...
Rule 'pdflatex':  Reasons for rerun
Changed files or newly in use/created:
  diff.aux
  diff.tex
Category 'changed_user':
  diff.tex

------------
Run number 1 of rule 'pdflatex'
------------
------------
Running 'pdflatex  -recorder  "diff.tex"'
------------
This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./diff.tex
LaTeX2e <2022-06-01> patch level 5
L3 programming layer <2022-12-17>
(/usr/share/texlive/texmf-dist/tex/latex/revtex/revtex4-2.cls
Document Class: revtex4-2 2020/10/03 4.2e (https://journals.aps.org/revtex/ for
 documentation)
ltxutil[2020/10/03 4.2e utilities package (portions licensed from W. E. Baxter 
web at superscript.com)]
ltxfront[2020/10/03 4.2e frontmatter package (AO,DPC,MD)]
ltxgrid[2020/10/03 4.2e page grid package (portions licen

Latexmk: Missing bbl file 'diff.bbl' in following:
 No file diff.bbl.


Latexmk: Found bibliography file(s):
  ./diffNotes.bib
  ./refs.bib
Latexmk: applying rule 'bibtex diff'...
Rule 'bibtex diff':  Reasons for rerun
Category 'other':
  Rerun of 'bibtex diff' forced or previously required

------------
Run number 1 of rule 'bibtex diff'
------------
------------
Running 'bibtex  "diff.aux"'
------------
This is BibTeX, Version 0.99d (TeX Live 2022)
The top-level auxiliary file: diff.aux
The style file: apsrev4-2.bst
Database file #1: diffNotes.bib
Database file #2: refs.bib
control{REVTEX42Control}, control.key{N/A}, control.author{N/A}, control.editor{N/A}, control.title{N/A}, control.pages{N/A}, control.year{N/A}, control.eprint{N/A}, 
control{apsrev42Control}, control.key{N/A}, control.author{08}, control.editor{1}, control.title{0}, control.pages{0}, control.year{1}, control.eprint{N/A}, 
Warning--jnrlst (dependency: not reversed) set 1
apsrev4-2.bst 2019-01-14 (MD) hand-edited version of apsrev4-1.bst
Control: key (0) 
Control: author (8) initials j

In [13]:
dir_new_full

'/home/ivan/Documents/GitHub/latexdiff-bot/test/new'

In [ ]:


# Create the diff.tex 
diff_file = 'diff.tex'
latexdiff_command = f'latexdiff {old_tex_file} {new_tex_file} > {dir_new_full}/{diff_file}'
subprocess.run(latexdiff_command, shell=True, check=True)

# Get the list of images used in the main LaTeX file
image_paths = []
with open(new_tex_file, 'r') as tex_file:
    for line in tex_file:
        if 'includegraphics' in line:
            start_index = line.find('{') + 1
            end_index = line.find('}')
            image_path = line[start_index:end_index]
            image_paths.append(image_path)

# Apply make_diff_image command to changed images
for image_path in image_paths:
    path_img_old = os.path.join(old_directory, image_path)
    path_img_new = os.path.join(new_directory, image_path)
    path_img_diff = os.path.join(new_directory, 'diff', image_path)
    make_diff_image(path_img_old, path_img_new, path_img_diff)

os.chdir('new')

# Compile the diff.tex file
pdflatex_command = f'latexmk -f -pdf {diff_file}'
subprocess.run(pdflatex_command, shell=True, check=True)

os.chdir('..')

shutil.move('new/diff.pdf', 'diff.pdf')



In [17]:
# Clean up temporary directories and files
shutil.rmtree('new')
shutil.rmtree('old')